In [1]:
from torch.utils.data.dataloader import DataLoader
from torch.nn.utils import rnn
import torch
from dataset import NottinghamDataset

In [2]:
# a, sr_a = librosa.load('data/nottingham-dataset/wav/ashover2.wav', sr=44100)
# b, sr_b = torchaudio.load('data/nottingham-dataset/wav/ashover2.wav')

In [3]:
# from scipy.io.wavfile import write
# import numpy as np

# write("example.wav", sr_a, a)

In [4]:
def collate_fn(data):
    data.sort(key=lambda d: d.shape[0], reverse=True)
    data_length = [d.shape[0] for d in data]
    data = rnn.pad_sequence(data, batch_first=True, padding_value=0)
    return data, data_length

In [5]:
dataset = NottinghamDataset('./data/nottingham-dataset/wav', 22050, transformation='mel_spectrogram')
dataloader = DataLoader(dataset, batch_size=3, collate_fn=collate_fn)

In [6]:
dataset[0].shape

torch.Size([1379, 64])

In [7]:
for i, (data, length) in enumerate(dataloader):
    if i > 0: break
    print(rnn.pack_padded_sequence(data, length, batch_first=True).data.shape)

torch.Size([8357, 64])


In [8]:
model = torch.nn.LSTM(64, 100, batch_first=True)

In [14]:
flag = 0
for i, (data, length) in enumerate(dataloader):
    if i > 0: break
    data = rnn.pack_padded_sequence(data, length, batch_first=True)
    output, hidden = model(data)
    if flag == 0:
        print(rnn.pad_packed_sequence(data, batch_first=True)[0].shape)
        output, out_len = rnn.pad_packed_sequence(output, batch_first=True)
        print(output.shape)
        print(output)
        flag = 1

torch.Size([3, 4135, 64])
torch.Size([3, 4135, 100])
tensor([[[-0.0301, -0.0494, -0.0059,  ...,  0.0797, -0.0499, -0.0579],
         [-0.1101, -0.0580, -0.0026,  ...,  0.0827, -0.3247, -0.0346],
         [-0.1662,  0.2955, -0.0074,  ...,  0.0556, -0.4317, -0.0351],
         ...,
         [-0.0107, -0.0258,  0.0578,  ..., -0.0588,  0.0342,  0.0494],
         [-0.0112, -0.0257,  0.0577,  ..., -0.0583,  0.0344,  0.0496],
         [-0.0122, -0.0245,  0.0580,  ..., -0.0579,  0.0340,  0.0490]],

        [[-0.0124, -0.0109,  0.0257,  ..., -0.0347,  0.0116,  0.0266],
         [-0.0159, -0.0169,  0.0393,  ..., -0.0497,  0.0187,  0.0399],
         [-0.0166, -0.0202,  0.0469,  ..., -0.0557,  0.0230,  0.0461],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.0615,  0.1019, -0.0229,  ...,  0.0169, -0.0345,  0

In [12]:
data.data.shape

torch.Size([1024048, 1])